In [1]:
import numba 
from numba import float64
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [2]:
@numba.jit(nopython=True)
def f_rl_eval(t,u,x,y,param):
    f =  np.zeros((1,1))
    
    i_l = x[0]
    
    L = param[0]
    R = param[1]

    v = u[0]

    di_l = 1.0/L*(v - R*i_l)
    
    f[0] = di_l

    return f

@numba.jit(nopython=True)
def h_rl_eval(t,u,x,y,param):
    h =  np.zeros((1,1))
    i_l = x[0]

    h[0,0] = i_l

    return h

@numba.jit(nopython=True)
def run():
    dt = 5e-6
    t_end = 0.1  
    decimation = 1
    N_steps = int(t_end/dt)
    N_saves = int(t_end/dt/decimation)
    T = np.zeros((N_saves,1))
    X = np.zeros((N_saves,1))
    
    t = 0.0
    x_0 = np.zeros((1,1))
    x = x_0

    X[0,:] = x
    
    param = np.array([0.001, # L
                      1.0   # R
                     ])


    u = np.zeros((1,1))

    it = 0
    it_decimation = 0
    it_save = 0
    solver = 1
    for it in range(N_steps):   
        if t > 0.01:
            u[0] = 10.0
            
        t += dt        

        if solver == 1:
            # solver trapezoidal 1 step
            x = x + dt*(f_rl_eval(t,u,x,0.0,param))
        
        if solver == 2:
            # solver trapezoidal 1 step
            f_1 = f_rl_eval(t,u,x,0.0,param)
            x_1 = x + dt*f_1
            x = x + 0.5*dt*(f_rl_eval(t,u,x,0.0,param)+ f_1)

        
        
        if it_decimation >= decimation:
            it_save += 1  
            X[it_save,:] = x
            T[it_save,:] = t
            it_decimation = 0
            
        it_decimation += 1              
        
        
    return T,X


#%timeit run()

T,X = run()



In [3]:
fig1 = figure(width=800, height=300)
fig1.line(x=T[:,0], y=X[:,0], color="red")
 


figs = gridplot([[fig1]])

show(figs)

In [4]:
#@numba.jit(float64[:](float64,float64,float64[:],float64,float64[:]))
@numba.jit(nopython=True)
def f_pi_eval(t,u,x,y,param):
    f =  np.zeros((1,1))
    xi = x[0]
    
    K_p = param[0]
    T_pi = param[1]

    ref = u[0]
    y   = u[1]

    error = ref - y
    dxi = error
    
    f[0] = dxi

    return f

@numba.jit(nopython=True)
def h_pi_eval(t,u,x,y,param):
    h =  np.zeros((1,1))
    
    xi = x[0]
    
    K_p = param[0]
    T_pi = param[1]

    ref = u[0]
    y   = u[1]

    error = ref - y
    
    h[0] = K_p*(error + 1.0/T_pi*xi)
    
    return h

@numba.jit(nopython=True)
def f_eval(t,u,x,y,param):
    
    f = np.zeros((2,1))
    u_pi= np.zeros((2,1))
    u_rl= np.zeros((2,1))
    
    ref = u[0]
    
    i_l = h_rl_eval(t,u,x[0,:],y,param)
    
    u_pi[0] = ref
    u_pi[1] = i_l
   
    pi_out = h_pi_eval(t,u_pi,x[1,:],y,param[2:4])
    
    
    u_rl[0] = pi_out
    
    f[0] = f_rl_eval(t,u_rl,x[0,:],y,param[0:2])
    f[1] = f_pi_eval(t,u_pi,x[1,:],y,param[2:4])
    
    return f
    
    
    
@numba.jit(nopython=True)
def run():
    dt = 5e-6
    t_end = 0.1  
    decimation = 10
    N_steps = int(t_end/dt)
    N_saves = int(t_end/dt/decimation)
    T = np.zeros((N_saves,1))
    X = np.zeros((N_saves,2))
    
    t = 0.0
    x_0 = np.zeros((2,1))
    x = x_0

    X[0,:] = x[:,0]
    
    param = np.array([0.001, # L
                      1.0,   # R
                      0.001/0.0001, # K_p
                      0.001   # T_pi
                     ])    

    u = np.zeros((2,1))

    it = 0
    it_decimation = 0
    it_save = 0
    solver = 1
    for it in range(N_steps):   
        if t > 0.01:
            u[0] = 10.0
            
        t += dt        

        if solver == 1:
            # solver trapezoidal 1 step
            x = x + dt*(f_eval(t,u,x,0.0,param))
        
        if solver == 2:
            # solver trapezoidal 1 step
            f_1 = f_eval(t,u,x,0.0,param)
            x_1 = x + dt*f_1
            x = x + 0.5*dt*(f_eval(t,u,x,0.0,param)+ f_1)

        
        
        if it_decimation >= decimation:
            it_save += 1  
            X[it_save,:] = x[:,0]
            T[it_save,:] = t
            it_decimation = 0
            
        it_decimation += 1              
        
        
    return T,X
   
%timeit run()

T,X = run()

The slowest run took 60.67 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 28.7 ms per loop


In [5]:
fig1 = figure(width=800, height=300)
fig1.line(x=T[:,0], y=X[:,0], color="red")
 


figs = gridplot([[fig1]])

show(figs)

In [6]:
np.zeros((2,1))

array([[ 0.],
       [ 0.]])

In [17]:
import ctrl
import electric

@numba.jit(nopython=True)
def f_eval(t,u,x,y,rl,pi):
    f = np.zeros((2,1))
    
    u_pi= np.zeros((2,1))
    u_rl= np.zeros((2,1))
    
    ref = u[0]
    
    i_l = rl.h_eval(t,u,x[0,:],y)
    
    u_pi[0] = ref
    u_pi[1] = i_l
   
    pi_out = pi.h_eval(t,u_pi,x[1,:],y)
    
    
    u_rl[0] = pi_out
    
    f[0] = rl.f_eval(t,u_rl,x[0,:],y)
    f[1] = pi.f_eval(t,u_pi,x[1,:],y)
    
    return f

   
    
@numba.jit(nopython=True)
def run():
    dt = 1e-6
    t_end = 0.1 
    decimation = 10
    N_steps = int(t_end/dt)
    N_saves = int(t_end/dt/decimation)
    T = np.zeros((N_saves,1))
    X = np.zeros((N_saves,2))
    
    t = 0.0
    x_0 = np.zeros((2,1))
    x = x_0

    X[0,:] = x[:,0]
    
    param = np.array([0.001, # L
                      1.0,   # R
                      0.001/0.0001, # K_p
                      0.001   # T_pi
                     ])    

    u = np.zeros((1,1))

    it = 0
    it_decimation = 0
    it_save = 0
    solver = 2

    
    
    f = np.zeros((2,1))
    
    pi = ctrl.pi()
    rl = electric.rl()
    
    pi.K_p = rl.L/1e-4
    pi.T_pi = rl.L/rl.R
    
    u_pi= np.zeros((2,1))
    u_rl= np.zeros((2,1))

    y = np.zeros(())
    
    for it in range(N_steps):   
        if t > 0.0:
            u[0] = 10.0
            
        t += dt        

        if solver == 1:
            # solver trapezoidal 1 step
            ref = u[0]

            i_l = rl.h_eval(t,u,x[0,:],y)

            u_pi[0] = ref
            u_pi[1] = i_l

            pi_out = pi.h_eval(t,u_pi,x[1,:],y)


            u_rl[0] = pi_out

            f[0] = rl.f_eval(t,u_rl,x[0,:],y)
            f[1] = pi.f_eval(t,u_pi,x[1,:],y)
    
    
            
            
            x = x + dt*(f)

        if solver == 11:
            # solver trapezoidal 1 step
            x = x + dt*(f_eval(t,u,x,y,rl,pi))
            
        if solver == 2:
            # solver trapezoidal 1 step
            f_1 = f_eval(t,u,x,y,rl,pi)
            x_1 = x + dt*f_1
            x = x + 0.5*dt*(f_eval(t,u,x_1,y,rl,pi)+ f_1)

        
        
        if it_decimation >= decimation:
            it_save += 1  
            X[it_save,:] = x[:,0]
            T[it_save,:] = t
            it_decimation = 0
            
        it_decimation += 1              
        
        
    return T,X
   
%timeit run()

T,X = run()

The slowest run took 5.78 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 245 ms per loop


In [18]:
fig1 = figure(width=800, height=300)
fig1.line(x=T[:,0], y=X[:,0], color="red")
figs = gridplot([[fig1]])

show(figs)

In [9]:
rl = electric.rl()
pi = ctrl.pi()
pi.K_p

10.0